In [1]:
import numpy as np
import random
import collections # to make a named tuple

In [ ]:
Coord = collections.namedtuple("Coord", ['x', 'y']) # create a named tuple with x and y parameters

In [2]:
def distance(pos1,pos2):
    distance = 0
    for i in range(len(pos1)):
        distance += (pos1[i]-pos2[i])**2
    return (distance)**0.5

In [3]:
distance((2,5),(4,8))

3.605551275463989

In [1]:
class Game:

    
    def __init__(self, size: int) -> None:
        # board of size nxn, team elements
        self.board = np.zeros(shape=(size,size))
        self.team = [[],[]]
        self.index = [0,0]
        self.size = size


    def get_unit(self, position: Coord) -> int:
        # gets unit at positon
        return self.board[position]
    
    def get_enemy_team(self, ind: int) -> int:
        # ind = player index 
        # if 0, self.team[-1] == self.team[1]
        # if 1, get self.team[0]
        return self.team[ind - 1]

    def add_unit(self, unit: int, position: Coord, team: int) -> None:


        if self.get_unit(position) != 0:
            raise Exception("Error: Invalid Position")
        
        # set the position and the team
        # multiply team by 1 or -1 depending on whether or not it's team 0 or 1
        self.board[position] = unit*[1,-1][team] 
        self.team[team].append(position)

# Tank units move like infantry units (towards closest enemy), except their kill percentage is halfed, 
# and their death percentage is halfed (death percentage to be implemented later).


    def get_closest_enemy(self, position: Coord, team: int) -> Coord:
        # get closest enemy
        enemy = []

        enemy_team = self.get_enemy_team(team)

        # append all enemies to a list
        for enemy_pos in enemy_team:
            enemy.append(distance(position, enemy_pos))
        # find closest enemy index
        enemy_ind = enemy.index(min(enemy))
        
        # get coordinates
        x, y = enemy_team[enemy_ind]
    
        return Coord(x ,y)
    

    def swap(self, position1, position2):
        self.board[position1], self.board[position2] = self.board[position2], self.board[position1]  
        return   

    def move_character(self, position: Coord, direction: str, units: int) -> Coord:
        # player_val = self.board[position]
        for _ in range(units):
            pos_x = position.x
            pos_y = position.y

            # direction dictionary 
            dir_dict = {"up"   : Coord(pos_x, pos_y + 1),
                        "down" : Coord(pos_x, pos_y - 1),
                        "left" : Coord(pos_x - 1, pos_y),
                        "right": Coord(pos_x + 1, pos_y)}

        # move in a certain direction n times or until some blockage is reached
        
            check_pos = dir_dict[direction]
            
            if self.board[check_pos] == 0   and \
                0 < check_pos.x < self.size and \
                0 < check_pos.y < self.size:
                self.swap(position, check_pos) # swap values
                position = check_pos

            else: 
                break

            return position

    def move_phase(self, position: Coord, enemy_pos: Coord,  id_int: int) -> Coord:

        # move left or right
        if enemy_pos.x != position.x:
            return self.move_character(position, "left") if enemy_pos.x < position.x else \
            self.move_character(position, "right")
        elif enemy_pos.y != position.y:
        # otherwise, move up or down
            return self.move_character(position, "down") if enemy_pos.y < position.y else \
            self.move_character(position, "up")

        # otherwise do nothing


    


    def tank_unit_ai(self, position: Coord) -> None:
         # MOVE PHASE: will move towards the closest enemy
        team = self.board[position] < 0
        
        enemy_pos = self.get_closest_enemy(position,team)
        
        new_pos = self.move_phase(position, enemy_pos)

        # if enemy_pos[0]!=position[0]:
        #     new_pos = (position[0]+[1,-1][position[0]>enemy_pos[0]],position[1])
        # elif enemy_pos[1]!=position[1]: 
        #     new_pos = (position[0], position[1]+[1,-1][position[1]>enemy_pos[1]])
        # else:
        #     new_pos = position
        # if self.board[new_pos] != 0:
        #     new_pos = position
        # else:
        #     self.board[position] = 0
        #     # tank unit will be displayed on the board by number 3
        #     self.board[new_pos] = [1,-1][team]*4
        
        # ACTION PHASE: killed smth, didn't kill smth, or can't kill smth
        enemy = []
        for enemy_pos in self.team[team-1]:
            enemy.append(distance(position, enemy_pos))
        # the closest enemy to the tank unit
        min_dist = min(enemy)
        if min_dist <= 1.1:
    # this just figures out which unit should be killed and the actual killing happens later on
    # kill percentage is half for tank
            if random.random() > 0.6:
                # print(team,'killed someone at',self.team[team-1][min(enumerate(enemy), key=lambda x: x[1])[0]])
                return new_pos, self.team[team-1][min(enumerate(enemy), key=lambda x: x[1])[0]]
        return new_pos, None
    
    def cavalry_ai(self,position):
        team = self.board[position] < 0
        
        enemy = []
        
        
        #getting the enemy position from my position
        for enemy_pos in self.team[team-1]:
            enemy.append(distance(position,enemy_pos))
        #self.team[indicates which team][indicates position?] 
        enemy_pos = self.team[team-1][min(enumerate(enemy),key=lambda x : x[1])[0]]
        
        #i assume position is a tuple
        #my unit will move twice
        temp_pos = [position[0],position[1]]
        new_pos = position
        for x in range(2):
            
            if enemy_pos[0]!=temp_pos[0]:
                #What determines if it goes foward or backwards depending on where enemy is
                temp_pos[0] = temp_pos[0]+[1,-1][temp_pos[0]>enemy_pos[0]]
                
            elif enemy_pos[1]!=temp_pos[1]: 
                temp_pos[1] = temp_pos[1]+[1,-1][temp_pos[1]>enemy_pos[1]]
                
            #print(temp_pos[0],temp_pos[1])
            if(self.board[(temp_pos[0],temp_pos[1])] == 0):
                new_pos = (temp_pos[0],temp_pos[1])
            

        #moves if there is no unit on it
        if self.board[new_pos] != 0:
            
            new_pos = position
        else:
            self.board[position] = 0
            self.board[new_pos] = [1,-1][team] * 3
            
            
        #updates new distances after move
        enemy = []
        for enemy_pos in self.team[team-1]:
            enemy.append(distance(position, enemy_pos))
        # print('enemy distances', enemy)
        min_dist = min(enemy)
        if min_dist <= 1.1:
            # print(team,'attempts to kill someone at',self.team[team-1][min(enumerate(enemy), key=lambda x: x[1])[0]])
            if random.random() > 0.3:
                # print(team,'killed someone at',self.team[team-1][min(enumerate(enemy), key=lambda x: x[1])[0]])
                return new_pos, self.team[team-1][min(enumerate(enemy), key=lambda x: x[1])[0]]
        return new_pos, None
            
    
    def default_ai(self, position):
        team = self.board[position] < 0
        # Move phase
        # nearest enemy troop
        enemy = []
        for enemy_pos in self.team[team-1]:
            enemy.append(distance(position, enemy_pos))
        enemy_pos = self.team[team-1][min(enumerate(enemy), key=lambda x: x[1])[0]]
        if enemy_pos[0]!=position[0]:
            new_pos = (position[0]+[1,-1][position[0]>enemy_pos[0]],position[1])
        elif enemy_pos[1]!=position[1]: 
            new_pos = (position[0], position[1]+[1,-1][position[1]>enemy_pos[1]])
        else:
            new_pos = position
        if self.board[new_pos] != 0:
            # don't move
            # print('cant move', self.board[new_pos], new_pos)
            new_pos = position
        else:
            self.board[position] = 0
            self.board[new_pos] = [1,-1][team]*1
        
        # action phase
        enemy = []
        for enemy_pos in self.team[team-1]:
            enemy.append(distance(position, enemy_pos))
        # print('enemy distances', enemy)
        min_dist = min(enemy)
        if min_dist <= 1.1:
            # print(team,'attempts to kill someone at',self.team[team-1][min(enumerate(enemy), key=lambda x: x[1])[0]])
            if random.random() > 0.2:
                # print(team,'killed someone at',self.team[team-1][min(enumerate(enemy), key=lambda x: x[1])[0]])
                return new_pos, self.team[team-1][min(enumerate(enemy), key=lambda x: x[1])[0]]
        return new_pos, None
    
    def ranged_ai(self, position):
        team = self.board[position] < 0
        # Move phase
        # nearest enemy troop
        enemy = []
        for enemy_pos in self.team[team-1]:
            enemy.append(distance(position, enemy_pos))
        enemy_pos = self.team[team-1][min(enumerate(enemy), key=lambda x: x[1])[0]]
        if enemy_pos[0]!=position[0]:
            new_pos = (position[0]+[1,-1][position[0]>enemy_pos[0]],position[1])
        elif enemy_pos[1]!=position[1]: 
            new_pos = (position[0], position[1]+[1,-1][position[1]>enemy_pos[1]])
        else:
            new_pos = position
        if self.board[new_pos] != 0:
            # don't move
            # print('cant move', self.board[new_pos], new_pos)
            new_pos = position
        else:
            self.board[position] = 0
            self.board[new_pos] = [1,-1][team]*2
        
        # action phase
        enemy = []
        for enemy_pos in self.team[team-1]:
            enemy.append(distance(position, enemy_pos))
        # print('enemy distances', enemy)
        min_dist = min(enemy)
        if min_dist < 4:
            # print(team,'attempts to kill someone at',self.team[team-1][min(enumerate(enemy), key=lambda x: x[1])[0]])
            if random.random() > 0.6:
                # print(team,'killed someone at',self.team[team-1][min(enumerate(enemy), key=lambda x: x[1])[0]])
                return new_pos, self.team[team-1][min(enumerate(enemy), key=lambda x: x[1])[0]]
        return new_pos, None
    
    def turn(self, team):
        unit_ai = [0,self.default_ai, self.ranged_ai, self.cavalry_ai, self.tank_unit_ai]
        # What is this doing?
        if self.index[team] == len(self.team[team]):
            if self.index[team-1] == len(self.team[team-1]):
                self.index = [0,0]
            else:
                return -1
        unit = int(abs(self.board[self.team[team][self.index[team]]]))
        # print('index',self.index[team], 'position',self.team[team][self.index[team]], 'team', team)
        new_position, death = unit_ai[unit](self.team[team][self.index[team]])
        self.team[team][self.index[team]] = new_position
        if death != None:
            self.board[death] = 0
            enemy_index = 0
            for i in range(len(self.team[team-1])):
                if death == self.team[team-1][i]:
                    enemy_index = i
                    break
            if self.index[team-1]>=enemy_index:
                self.index[team-1]-=1
            self.team[team-1].pop(enemy_index)
            # print(self.team[team-1], 'is left over')
            if len(self.team[team-1]) == 0:
                return team
        self.index[team]+=1
        return -1
    
    def run(self):
        turns = 0
        while(1):
            for team in [0,1]:
                turns += 1
                win = self.turn(team)
                if win != -1:
                    return win
                
    def create_board(self, board):
        pieces = np.transpose(np.nonzero(board))
        #print(pieces)
        for piece in pieces:
            piece = tuple(piece)
            #print(piece)
            self.add_unit(abs(board[piece]), piece, board[piece]<0)

    def __repr__(self):
        s = "+ | "
        

        axis_range = range(self.size)
        
        for i in axis_range:
            s += str(i) + " | "
        s += "\n"
        for y in axis_range:
            s += "_"*(4*self.size + 3) + "\n\n"
            s += str(y) + " |"

            for x in range(self.size):
                s += " " + str(int(self.board[Coord(x, y  )])) + " |"
            s += "\n"


        return s
    

        


    

    


In [ ]:
- 1, position_y)]
                if check_pos == 0:
                    self.swap(position, check_pos)
                else: break # no need to repeat loop

            elif direction == "right":
                

            







    def move_phase(self, position, id_int):
        pass 


    def tank_unit_ai(self, position):
         # MOVE PHASE: will move towards the closest enemy
        team = self.board[position] < 0
        
        enemy_pos = self.get_closest_enemy(position,team)
        
        if enemy_pos[0]!=position[0]:
            new_pos = (position[0]+[1,-1][position[0]>enemy_pos[0]],position[1])
        elif enemy_pos[1]!=position[1]: 
            new_pos = (position[0], position[1]+[1,-1][position[1]>enemy_pos[1]])
        else:
            new_pos = position
        if self.board[new_pos] != 0:
            new_pos = position
        else:
            self.board[position] = 0
            # tank unit will be displayed on the board by number 3
            self.board[new_pos] = [1,-1][team]*4
        
        # ACTION PHASE: killed smth, didn't kill smth, or can't kill smth
        enemy = []
        for enemy_pos in self.team[team-1]:
            enemy.append(distance(position, enemy_pos))
        # the closest enemy to the tank unit
        min_dist = min(enemy)
        if min_dist <= 1.1:
    # this just figures out which unit should be killed and the actual killing happens later on
    # kill percentage is half for tank
            if random.random() > 0.6:
                # print(team,'killed someone at',self.team[team-1][min(enumerate(enemy), key=lambda x: x[1])[0]])
                return new_pos, self.team[team-1][min(enumerate(enemy), key=lambda x: x[1])[0]]
        return new_pos, None
    
    def cavalry_ai(self,position):
        team = self.board[position] < 0
        
        enemy = []
        
        
        #getting the enemy position from my position
        for enemy_pos in self.team[team-1]:
            enemy.append(distance(position,enemy_pos))
        #self.team[indicates which team][indicates position?] 
        enemy_pos = self.team[team-1][min(enumerate(enemy),key=lambda x : x[1])[0]]
        
        #i assume position is a tuple
        #my unit will move twice
        temp_pos = [position[0],position[1]]
        new_pos = position
        for x in range(2):
            
            if enemy_pos[0]!=temp_pos[0]:
                #What determines if it goes foward or backwards depending on where enemy is
                temp_pos[0] = temp_pos[0]+[1,-1][temp_pos[0]>enemy_pos[0]]
                
            elif enemy_pos[1]!=temp_pos[1]: 
                temp_pos[1] = temp_pos[1]+[1,-1][temp_pos[1]>enemy_pos[1]]
                
            #print(temp_pos[0],temp_pos[1])
            if(self.board[(temp_pos[0],temp_pos[1])] == 0):
                new_pos = (temp_pos[0],temp_pos[1])
            

        #moves if there is no unit on it
        if self.board[new_pos] != 0:
            
            new_pos = position
        else:
            self.board[position] = 0
            self.board[new_pos] = [1,-1][team] * 3
            
            
        #updates new distances after move
        enemy = []
        for enemy_pos in self.team[team-1]:
            enemy.append(distance(position, enemy_pos))
        # print('enemy distances', enemy)
        min_dist = min(enemy)
        if min_dist <= 1.1:
            # print(team,'attempts to kill someone at',self.team[team-1][min(enumerate(enemy), key=lambda x: x[1])[0]])
            if random.random() > 0.3:
                # print(team,'killed someone at',self.team[team-1][min(enumerate(enemy), key=lambda x: x[1])[0]])
                return new_pos, self.team[team-1][min(enumerate(enemy), key=lambda x: x[1])[0]]
        return new_pos, None
            
    
    def default_ai(self, position):
        team = self.board[position] < 0
        # Move phase
        # nearest enemy troop
        enemy = []
        for enemy_pos in self.team[team-1]:
            enemy.append(distance(position, enemy_pos))
        enemy_pos = self.team[team-1][min(enumerate(enemy), key=lambda x: x[1])[0]]
        if enemy_pos[0]!=position[0]:
            new_pos = (position[0]+[1,-1][position[0]>enemy_pos[0]],position[1])
        elif enemy_pos[1]!=position[1]: 
            new_pos = (position[0], position[1]+[1,-1][position[1]>enemy_pos[1]])
        else:
            new_pos = position
        if self.board[new_pos] != 0:
            # don't move
            # print('cant move', self.board[new_pos], new_pos)
            new_pos = position
        else:
            self.board[position] = 0
            self.board[new_pos] = [1,-1][team]*1
        
        # action phase
        enemy = []
        for enemy_pos in self.team[team-1]:
            enemy.append(distance(position, enemy_pos))
        # print('enemy distances', enemy)
        min_dist = min(enemy)
        if min_dist <= 1.1:
            # print(team,'attempts to kill someone at',self.team[team-1][min(enumerate(enemy), key=lambda x: x[1])[0]])
            if random.random() > 0.2:
                # print(team,'killed someone at',self.team[team-1][min(enumerate(enemy), key=lambda x: x[1])[0]])
                return new_pos, self.team[team-1][min(enumerate(enemy), key=lambda x: x[1])[0]]
        return new_pos, None
    
    def ranged_ai(self, position):
        team = self.board[position] < 0
        # Move phase
        # nearest enemy troop
        enemy = []
        for enemy_pos in self.team[team-1]:
            enemy.append(distance(position, enemy_pos))
        enemy_pos = self.team[team-1][min(enumerate(enemy), key=lambda x: x[1])[0]]
        if enemy_pos[0]!=position[0]:
            new_pos = (position[0]+[1,-1][position[0]>enemy_pos[0]],position[1])
        elif enemy_pos[1]!=position[1]: 
            new_pos = (position[0], position[1]+[1,-1][position[1]>enemy_pos[1]])
        else:
            new_pos = position
        if self.board[new_pos] != 0:
            # don't move
            # print('cant move', self.board[new_pos], new_pos)
            new_pos = position
        else:
            self.board[position] = 0
            self.board[new_pos] = [1,-1][team]*2
        
        # action phase
        enemy = []
        for enemy_pos in self.team[team-1]:
            enemy.append(distance(position, enemy_pos))
        # print('enemy distances', enemy)
        min_dist = min(enemy)
        if min_dist < 4:
            # print(team,'attempts to kill someone at',self.team[team-1][min(enumerate(enemy), key=lambda x: x[1])[0]])
            if random.random() > 0.6:
                # print(team,'killed someone at',self.team[team-1][min(enumerate(enemy), key=lambda x: x[1])[0]])
                return new_pos, self.team[team-1][min(enumerate(enemy), key=lambda x: x[1])[0]]
        return new_pos, None
    
    def turn(self, team):
        unit_ai = [0,self.default_ai, self.ranged_ai, self.cavalry_ai, self.tank_unit_ai]
        # What is this doing?
        if self.index[team] == len(self.team[team]):
            if self.index[team-1] == len(self.team[team-1]):
                self.index = [0,0]
            else:
                return -1
        unit = int(abs(self.board[self.team[team][self.index[team]]]))
        # print('index',self.index[team], 'position',self.team[team][self.index[team]], 'team', team)
        new_position, death = unit_ai[unit](self.team[team][self.index[team]])
        self.team[team][self.index[team]] = new_position
        if death != None:
            self.board[death] = 0
            enemy_index = 0
            for i in range(len(self.team[team-1])):
                if death == self.team[team-1][i]:
                    enemy_index = i
                    break
            if self.index[team-1]>=enemy_index:
                self.index[team-1]-=1
            self.team[team-1].pop(enemy_index)
            # print(self.team[team-1], 'is left over')
            if len(self.team[team-1]) == 0:
                return team
        self.index[team]+=1
        return -1
    
    def run(self):
        turns = 0
        while(1):
            for team in [0,1]:
                turns += 1
                win = self.turn(team)
                if win != -1:
                    return win
                
    def create_board(self, board):
        pieces = np.transpose(np.nonzero(board))
        #print(pieces)
        for piece in pieces:
            piece = tuple(piece)
            #print(piece)
            self.add_unit(abs(board[piece]), piece, board[piece]<0)

        


    

    


In [74]:
def random_board(size=10,units=[1,2,3,4],max_units=6):
    board = np.zeros(shape=(size,size))
    # team0_board = board[:,:size//2]
    # team1_board = board[:,size//2:]
    for unit in units:
        num_units = round(random.random()*max_units)
        for _ in range(num_units):
            while(1):
                pos = (int(random.random()*size), int(random.random()*size//2))
                if board[pos]!=0:
                    continue
                board[pos] = unit
                break
        num_units = round(random.random()*max_units)
        for _ in range(num_units):
            while(1):
                pos = (int(random.random()*size), int(random.random()*size//2)+size//2)
                if board[pos]!=0:
                    continue
                board[pos] = -1*unit
                break
    if np.sum(board>0) == 0 or np.sum(board<0) == 0:
        board = random_board(size,units,max_units)
    return board

In [100]:
size = 10
# board = np.zeros((size, size))
board = random_board()
print(board)
winners = []
for i in range(1000):
    game = Game(size)
    game.create_board(board)
    winners.append(game.run())
print(np.mean(winners), np.sum(winners))

[[ 0.  0.  2.  0.  0.  0.  0. -3.  0.  0.]
 [ 3.  4.  1.  0.  0.  0. -2. -2.  0. -4.]
 [ 0.  4.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 3.  0.  0.  2.  1.  0.  0.  0.  0.  0.]
 [ 1.  0.  0.  3.  3. -2.  0.  0.  0.  0.]
 [ 0.  0.  4.  2.  0.  0.  0.  0.  0.  0.]
 [ 1.  0.  3.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  2.  0.  0.  0.  0.  0.]
 [ 2.  1.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  4.  0.  0.  0. -3.  0.  0.  0.  0.]]


C:\Users\co41c\AppData\Local\Temp\ipykernel_16268\3819378594.py:10: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  self.board[position] = unit*[1,-1][team]
C:\Users\co41c\AppData\Local\Temp\ipykernel_16268\3819378594.py:11: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  self.team[team].append(position)
C:\Users\co41c\AppData\Local\Temp\ipykernel_16268\3819378594.py:147: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  new_pos = (position[0]+[1,-1][position[0]>enemy_pos[0]],position[1])
C:\Users\co41c\AppData\Local\Temp\ipykernel_16268\3819378594.py:70: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  temp_pos[0] = temp_pos[0]+[1,-1][temp_pos[0]>enemy_pos[0]]
C:\Users\co41c\AppData\Local\Temp\ipykernel_16268\3819378594.py:86: DeprecationWarning: In future, it

0.002 2


In [101]:
def new_run(fn, size=10, runs=100, max_units=6):
    board = random_board(size=size, max_units=max_units)
    # print(board)
    winners = []
    for _ in range(runs):
        game = Game(size)
        game.create_board(board)
        result = game.run()
        # print(result)
        winners.append(result)
    # print(winners)
    # print(np.sum(winners), np.mean(winners))
    result = np.mean(winners, axis=0)
    with open(fn, 'a', encoding='utf-8') as my_file:
        content = ''
        for i in board:
            for j in i:
                content+=str(j)+','
        content+=str(result)+'\n'
        my_file.write(content)

In [103]:
for _ in range(1000):
    new_run('results.csv')

C:\Users\co41c\AppData\Local\Temp\ipykernel_16268\3819378594.py:10: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  self.board[position] = unit*[1,-1][team]
C:\Users\co41c\AppData\Local\Temp\ipykernel_16268\3819378594.py:11: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  self.team[team].append(position)
C:\Users\co41c\AppData\Local\Temp\ipykernel_16268\3819378594.py:70: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  temp_pos[0] = temp_pos[0]+[1,-1][temp_pos[0]>enemy_pos[0]]
C:\Users\co41c\AppData\Local\Temp\ipykernel_16268\3819378594.py:86: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  self.board[new_pos] = [1,-1][team] * 3
C:\Users\co41c\AppData\Local\Temp\ipykernel_16268\3819378594.py:26: DeprecationWarning: In future, it will be an error for 'np.bool_